In [ ]:
# Useful starting lines
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
# from plots import visualization
from implementations import *
from proj1_helpers import *
from helpers import equalize_predictions

%load_ext autoreload
%autoreload 2

## 1. Load boson data 

In [ ]:
# just load the training dataset
data_path = "../dataset/train.csv"
y_loaded, x_loaded, ids_te = load_csv_data(data_path, sub_sample=False)
y_loaded = y_loaded.reshape((-1, 1))
y_loaded.shape, x_loaded.shape

## 2. Clean data

In [ ]:
x_loaded.shape, y_loaded.shape

In [ ]:
x_all, y_all = clean_input_data(x_loaded.copy(), y_loaded.copy(), corr=1)
for i in range(len(y_all)):
    y_all[i][y_all[i]== -1] = 0
x_all[0].shape, x_all[1].shape, x_all[2].shape, x_all[3].shape

In [ ]:
y_all[0].shape, y_all[1].shape, y_all[2].shape, y_all[3].shape

In [ ]:
# check itis 250000
x_all[0].shape[0] + x_all[1].shape[0] +  x_all[2].shape[0] + x_all[3].shape[0]

In [ ]:
for i in range(4):
    print(np.sum(np.isnan(x_all[i])))

### 2.9 Choose the degree and set the gamma

In [ ]:
i = 3
# choose degree
degree = 3
#tx_sub = build_poly(x_sub, degree)
tx_all = build_poly(x_all[i], degree)
tx_all.shape, y_all[i].shape

In [ ]:
ncolumns = x_all[i].shape[1]

# 50000 data, 14 columns, 999=mean, 0=mean
gamma = np.concatenate([
     # gamma for constant and 1st degree 
    np.ones(ncolumns+1)*1e-5,
    # gammma 2nd degree
    np.ones(ncolumns)*1e-6, 
    # gamma for 3rd degree 
    np.ones(ncolumns)*1e-7,
#     # gamma for 4th degree
#     np.ones(ncolumns)*1e-10,
#     # gamma for 5th degree
#     np.ones(ncolumns)*1e-12,
#     # gamma for 6th degree
#     np.ones(ncolumns)*1e-15,
#     # gamma for 7th degree 
#     np.ones(ncolumns)*1e-17,
#     # gamma for 8th degree 
#     np.ones(ncolumns)*1e-20,
#     # gamma for 9th degree 
#     np.ones(ncolumns)*1e-24,
#     # gamma for 10th degree 
#     np.ones(ncolumns)*1e-24,
#     # gamma for 11th degree 
#     np.ones(ncolumns)*1e-28,
])\
.reshape((-1, 1))*0.05
gamma.shape

In [ ]:
# for clean_x3
n_bool_cols = bool_cols.shape[1]
n_other_cols = x_all.shape[1]

# 50000 data, 14 columns, 999=mean, 0=mean
gamma = np.concatenate([
    # gamma for bool cols
    np.ones(n_bool_cols)*1e-5,
     # gamma for constant and 1st degree 
    np.ones(n_other_cols+1)*1e-5,
    # gammma 2nd degree
    np.ones(n_other_cols)*1e-6, 
    # gamma for 3rd degree 
    np.ones(n_other_cols)*1e-7,
    # gamma for 4th degree
    np.ones(n_other_cols)*1e-9,
    # gamma for 5th degree
    np.ones(n_other_cols)*1e-13,
#     # gamma for 6th degree
#     np.ones(n_other_cols)*1e-16,
#     # gamma for 7th degree 
#     np.ones(n_other_cols)*1e-19,
#     # gamma for 8th degree 
#     np.ones(n_other_cols)*1e-20,
#     # gamma for 9th degree 
#     np.ones(n_other_cols)*1e-24,
#     # gamma for 10th degree 
#     np.ones(n_other_cols)*1e-24,
#     # gamma for 11th degree 
#     np.ones(n_other_cols)*1e-28,
])\
.reshape((-1, 1))*0.5

gamma.shape

## 3. Train the model

### Logistic Regression

In [ ]:
initial_w = np.zeros((tx.shape[1], 1))
max_iters = 142
ouptut_step = 100
loss, w = gradient_descent(y_all[i], tx_all, initial_w, max_iters, gamma, num_batches=1,
                           plot_losses=True, print_output=True, ouptut_step=50, costfunc=CostFunction.LIKELIHOOD)

In [ ]:
compute_loss(y_all[i], tx_all, w, costfunc=CostFunction.SUCCESS_RATIO)

### Penalized logistic regression

In [ ]:
initial_w = np.zeros((tx.shape[1], 1))
max_iters = 500
ouptut_step = 100
lambda_ = 1
loss, w = gradient_descent(y_all[i], tx_all, initial_w, max_iters, gamma, num_batches=1, lambda_=lambda_,
                           plot_losses=True, print_output=True, ouptut_step=50, costfunc=CostFunction.LIKELIHOOD)

In [ ]:
compute_loss(y_all[i], tx_all, w, costfunc=CostFunction.SUCCESS_RATIO)

## 4. Create submit file

##### a. Load test data

In [ ]:
data_path = "../dataset/test.csv"
y_te_loaded, x_te_loaded, ids_te_loaded = load_csv_data(data_path, sub_sample=False)
y_te_loaded.shape, x_te_loaded.shape

#### b. Clean

In [ ]:
ids_te[0].shape

In [ ]:
# clean, have to split the ids too otherwise we lose the mappings
x_te, ids_te = clean_input_data(x_te_loaded.copy(), ids_te_loaded.copy())

#### c. Build the polynomial

In [ ]:
# build
degree = 3
tx_te = []
for x_te_ in x_te:
    tx_te.append(build_poly(x_te_, degree))
tx_te[0].shape, tx_te[1].shape, tx_te[2].shape, tx_te[3].shape

####  d. Predict 

In [ ]:
y_te_pred = []
for jet in range(4):
    y_te_pred.append(predict_labels(weigths[jet], tx_te[jet]))
y_te_pred[0].shape, y_te_pred[1].shape, y_te_pred[2].shape, y_te_pred[3].shape

In [ ]:
# concatenate all the results
for jet in range(4):
    ids_te[jet] = ids_te[jet].reshape((-1, 1))
y_pred = np.row_stack([y_te_pred[0], y_te_pred[1], y_te_pred[2], y_te_pred[3]])
ids = np.row_stack([ids_te[0], ids_te[1], ids_te[2], ids_te[3]])
y_pred.shape, ids.shape

In [ ]:
# check it makes sense
(y_pred==-1).sum(), (y_pred==1).sum()
# (393191, 175047)

#### e. Store predictions

In [ ]:
# store the predictions
sub_file_name = "logistic_regression_definitiveClean_degree3"
create_csv_submission(ids, y_pred, sub_file_name)

## Store/load weights

### Store

In [ ]:
weigth_name = "w3"
#file_path = "weigths/no_perc/w1"
file_path = "weigths/with_perc3/" + weigth_name
json.dump(w.tolist(), codecs.open(file_path, 'w', encoding='utf-8'), separators=(',', ':'), sort_keys=True, indent=4)

### Load

In [ ]:
weigths = []
for weigth_name in ["w0", "w1", "w2", "w3"]:
    file_path = "weigths/with_perc3/"+weigth_name
    obj_text = codecs.open(file_path, 'r', encoding='utf-8').read()
    weigths.append(np.array(json.loads(obj_text)))
weigths[0].shape, weigths[1].shape, weigths[2].shape, weigths[3].shape